In [1]:
import torch 
import torch.nn as nn 

from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 

import numpy as np 
import matplotlib.pyplot as plt

For bulding a model we always follow 3 key steps: 

1. Design of the model (input/output sizes, forward pass)
2. Construct Loss and Optimizer 
3. Train loop (forward pass for pred and loss, backward pass, update weights)

In [2]:
### 0- Prepare data ###
X_numpy, y_numpy = datasets.load_breast_cancer(return_X_y=True)

n_samples, n_features = X_numpy.shape

# train and test set 
X_train, X_test, y_train, y_test = train_test_split(X_numpy, y_numpy,
                                                    test_size=0.2,
                                                    random_state=77)

# scaling the data 
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

# transform to  column vectors
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

# print shapes 
print(X_train.shape)
print(y_train.shape)
print('')
print(X_test.shape)
print(y_test.shape)

torch.Size([455, 30])
torch.Size([455, 1])

torch.Size([114, 30])
torch.Size([114, 1])


In [3]:
### 1- Model ###

class MiRegresionLogistica(nn.Module):

    def __init__(self, n_input_features):
        super(MiRegresionLogistica, self).__init__()

        # one linear layer 
        self.linear = nn.Linear(n_input_features, 1)
    
    def forward(self, x):
        
        y_pred = torch.sigmoid(self.linear(x))

        return y_pred 


In [4]:
model = MiRegresionLogistica(n_features)

# Binary Cross Entropy Loss 
criterion = nn.BCELoss()

learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(),
                            lr = learning_rate)

In [5]:
max_epcohs = 100 

for epoch in range(max_epcohs):

    y_pred = model(X_train)

    loss = criterion(y_pred, y_train)

    loss.backward()

    optimizer.step()

    optimizer.zero_grad()

    if (epoch+1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss = {loss:.4f}')

epoch: 10, loss = 0.2401
epoch: 20, loss = 0.1806
epoch: 30, loss = 0.1539
epoch: 40, loss = 0.1381
epoch: 50, loss = 0.1275
epoch: 60, loss = 0.1197
epoch: 70, loss = 0.1137
epoch: 80, loss = 0.1089
epoch: 90, loss = 0.1049
epoch: 100, loss = 0.1016


In [6]:
with torch.no_grad():
    y_pred = model(X_test)
    y_pred_cls = y_pred.round()
    acc = y_pred_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy = {acc:.4f}')

accuracy = 0.9737
